In [127]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Markdown

from funcs.feature_engineering import Preprocessing, FeatureSelection
from funcs.clustering import Clustering
from funcs.integration_exploration import Integration, Exploration

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

In [128]:
path = ''

In [129]:
metricFeatures: list[str] = ['Application order','Previous qualification score','Entry score','Age at enrollment','N units credited 1st period',
       'N units taken 1st period', 'N scored units 1st period',
       'N units approved 1st period', 'Average grade 1st period',
       'N unscored units 1st period', 'N units credited 2nd period',
       'N units taken 2nd period', 'N scored units 2nd period',
       'N units approved 2nd period', 'Average grade 2nd period',
       'N unscored units 2nd period','Social Popularity']
categoricalFeatures: list[str] =  ['Application mode','Marital status','Course','Previous qualification','Nationality','Mother\'s qualification',
       'Father\'s qualification','Mother\'s occupation','Father\'s occupation']
boolFeatures: list[str] = ['Morning shift participation','Displaced','Special needs','Debtor','Regularized Fees','Gender_Male','External Funding','International']

In [130]:
academicFeatures: list[str] = [
    'Entry score',
    'Average scored units',
    'Average grades',
    'Average units taken',
    'Average units approved',
    'Average units credited',
    'Average unscored units',
    'Success'
]

demographicFeatures: list[str] = [
    'Age at enrollment',
    'Marital status',
    'Course',
    'Previous qualification',
    'Nationality',
    'Mother\'s qualification',
    'Father\'s qualification',
    'Mother\'s occupation',
    'Father\'s occupation',
    'Special needs',
    'International',
    'Debtor',
    'Regularized Fees',
    'Gender_Male'
]

In [131]:
def runAll(
    metricFeatures: list[str],
    categoricalFeatures: list[str],
    boolFeatures: list[str],
    academicFeatures: list[str],
    demographicFeatures: list[str],
    removedAcademicFeatures: list[str] | list = [],
    removedDemographicFeatures: list[str] | list = [],
    path: str = '',
    *,
    describeDF: bool = False,
    featureSelection: bool = False,
    getPairPlot: bool = False,
    elbowGraph: bool = True, 
    silhouetteGraph: bool = True, 
    dendrogram: bool = True,
    clusterProfiling: bool = False,
    nClustersAcademic: int = 0,
    nClustersDemographic: int = 0,
) -> None:
    """_summary_

    Args:
        metricFeatures (list[str]): _description_
        categoricalFeatures (list[str]): _description_
        boolFeatures (list[str]): _description_
        academicFeatures (list[str]): _description_
        demographicFeatures (list[str]): _description_
        removedAcademicFeatures (list[str] | list, optional): _description_. Defaults to [].
        removedDemographicFeatures (list[str] | list, optional): _description_. Defaults to [].
        path (str, optional): _description_. Defaults to ''.
        describeDF (bool, optional): _description_. Defaults to False.
        featureSelection (bool, optional): _description_. Defaults to False.
        getPairPlot (bool, optional): _description_. Defaults to False.
        elbowGraph (bool, optional): _description_. Defaults to True.
        silhouetteGraph (bool, optional): _description_. Defaults to True.
        dendrogram (bool, optional): _description_. Defaults to True.
        clusterProfiling (bool, optional): _description_. Defaults to False.
        nClustersAcademic (int, optional): _description_. Defaults to 0.
        nClustersDemographic (int, optional): _description_. Defaults to 0.
    """
    dataDF: pd.DataFrame = Integration.importdata(path)
    dataAcademic: pd.DataFrame
    dataDemographic: pd.DataFrame
    scaler: MinMaxScaler
    academicScaler: MinMaxScaler
    demographicScaler: MinMaxScaler

    if describeDF:
        display(Exploration.describeData(dataDF, metricFeatures, categoricalFeatures))
        return

    dataDF, dataAcademic, dataDemographic, scaler, academicScaler, demographicScaler = Preprocessing.runPreprocessing(dataDF, metricFeatures, boolFeatures, academicFeatures, demographicFeatures, removedAcademicFeatures, removedDemographicFeatures, grouping="high")

    if featureSelection:
        if getPairPlot:
            display(FeatureSelection.pairPlots(dataAcademic, "Academic"))
            display(FeatureSelection.pairPlots(dataDemographic, "Demographic"))

        display(FeatureSelection.checkCorr(dataAcademic))
        display(FeatureSelection.checkCorr(dataDemographic))
        return

    if nClustersAcademic == 0:
        Clustering.somWrapper(dataAcademic, "Academic", nClustersAcademic)
        return
    
    dataAcademic = Clustering.somWrapper(dataAcademic, "Academic", nClustersAcademic)

    if nClustersDemographic == 0:
        Clustering.runKMeans(dataDemographic,nClustersDemographic, elbowGraph=elbowGraph, silhouetteGraph=silhouetteGraph, dendrogram=dendrogram)
        return
    
    dataDemographic = Clustering.runKMeans(dataDemographic,nClustersDemographic, elbowGraph=elbowGraph, silhouetteGraph=silhouetteGraph, dendrogram=dendrogram)

    if clusterProfiling:
        display(Markdown("### Academic Perspective"))
        FeatureSelection.clusterProfiles(dataAcademic, ["label"], (32, 10))
        FeatureSelection.getVariableClusterGraphs(dataAcademic, "label")

        display(Markdown("### Demographic Perspective"))
        FeatureSelection.clusterProfiles(dataDemographic, ["label"], (32, 10))
        FeatureSelection.getVariableClusterGraphs(dataDemographic, "label")

        display(Markdown("### Final Results"))
        FeatureSelection.clusterProfiles(Preprocessing.scaleData(Clustering.mergePerspectives(dataDF, dataAcademic, dataDemographic, scaler))[0], ["final_groups"], (50, 10))
        FeatureSelection.getVariableClusterGraphs((Clustering.mergePerspectives(dataDF, dataAcademic, dataDemographic, scaler)), "final_groups")
        return

    display(Markdown("### Academic Perspective"))
    tempDF = dataAcademic.drop("label", axis=1)
    tempDF = pd.DataFrame(academicScaler.inverse_transform(tempDF), index=tempDF.index, columns=tempDF.columns)
    tempDF = tempDF.join(dataAcademic["label"])
    display(tempDF.groupby(['label']).describe().T)

    display(Markdown("### Demographic Perspective"))
    tempDF = dataDemographic.drop("label", axis=1)
    tempDF = pd.DataFrame(demographicScaler.inverse_transform(tempDF), index=tempDF.index, columns=tempDF.columns)
    tempDF = tempDF.join(dataDemographic["label"])
    display(tempDF.groupby(['label']).describe().T)
    
    display(Markdown("### Final Results"))
    display(Clustering.mergePerspectives(dataDF, dataAcademic, dataDemographic, scaler).groupby(['final_groups']).describe().T)

In [132]:
removedAcademicFeatures: list[str] = [
    'Average units taken',
    'Average units credited',
    'Average unscored units'
]

removedDemographicFeatures: list[str] = [
    'Special needs',
    'International',
    'Debtor',
    'Regularized Fees',
    'Marital status-together',
    'Marital status-nan',
    'Father\'s qualification-None',
    'Mother\'s occupation-Service Industry',
    'Mother\'s occupation-White collar Jobs',
    'Father\'s occupation-Professional Fields',
    'Father\'s occupation-Service Industry',
    'Father\'s occupation-Technical and Skilled Trades',
    'Father\'s occupation-White collar Jobs',
    'Father\'s occupation-nan',
    'Mother\'s qualification-None',
    'Mother\'s qualification-nan',
    'Mother\'s occupation-Technical and Skilled Trades',
    'Mother\'s occupation-nan',
    'Father\'s occupation-Security and Armed Forces',
    'Father\'s qualification-nan',
    'Nationality-Angolan',
    'Nationality-Brazilian',
    'Nationality-Cape Verdean',
    'Nationality-Colombian',
    'Nationality-Cuban',
    'Nationality-Dutch',
    'Nationality-English',
    'Nationality-German',
    'Nationality-Guinean',
    'Nationality-Italian',
    'Nationality-Lithuanian',
    'Nationality-Mexican',
    'Nationality-Moldova',
    'Nationality-Mozambican',
    'Nationality-Portuguese',
    'Nationality-Romanian',
    'Nationality-Russian',
    'Nationality-Santomean',
    'Nationality-Spanish',
    'Nationality-Turkish',
    'Nationality-Ukrainian',
    'Nationality-nan',
    'Course-Advertising and Marketing Management',
    'Course-Agronomy',
    'Course-Animation and Multimedia Design',
    'Course-Basic Education',
    'Course-Biofuel Production Technologies',
    'Course-Communication Design',
    'Course-Equinculture',
    'Course-Informatics Engineering',
    'Course-Journalism and Communication',
    'Course-Management',
    'Course-Management (evening attendance)',
    'Course-Nursing',
    'Course-Oral Hygiene',
    'Course-Social Service',
    'Course-Social Service (evening attendance)',
    'Course-Tourism',
    'Course-Veterinary Nursing',
    'Course-nan',
    'Previous qualification-Higher',
    'Previous qualification-Lower',
    'Previous qualification-Middle',
    'Previous qualification-nan'
]

In [134]:
runAll(
    metricFeatures,
    categoricalFeatures,
    boolFeatures,
    academicFeatures,
    demographicFeatures,
    removedAcademicFeatures,
    removedDemographicFeatures,
    path=path,
    describeDF=False,
    featureSelection=False,
    getPairPlot=False,
    elbowGraph=True,
    silhouetteGraph=True,
    dendrogram=True,
    clusterProfiling=False,
    nClustersAcademic=3,
    nClustersDemographic=3
)

e:\GitHub\mlproj\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


### Academic Perspective

label                                   0           1            2
Entry score            count  2106.000000  816.000000  1323.000000
                       mean    128.158879  126.325245   125.426228
                       std      14.107065   16.621587    13.351658
                       min      95.000000   95.000000    95.000000
                       25%     119.100000  115.425000   117.000000
                       50%     127.000000  124.800000   124.300000
                       75%     135.700000  136.100000   133.000000
                       max     190.000000  183.500000   190.000000
Average scored units   count  2106.000000  816.000000  1323.000000
                       mean      7.984330    3.811275     9.377929
                       std       2.201488    3.793025     2.376294
                       min       0.000000    0.000000     0.000000
                       25%       6.500000    0.000000     8.000000
                       50%       8.000000    4.250000     9.500000
                       75%       9.000000    7.000000    11.000000
                       max      17.500000   15.500000    16.500000
Average grades         count  2106.000000  816.000000  1323.000000
                       mean     13.114141    0.995632    11.793768
                       std       1.185156    2.198919     1.842125
                       min       6.142857    0.000000     5.000000
                       25%      12.300000    0.000000    11.244048
                       50%      13.016429    0.000000    11.920833
                       75%      13.916667    0.000000    12.750000
                       max      18.283654    8.000000    17.166667
Average units approved count  2106.000000  816.000000  1323.000000
                       mean      6.204416    0.291667     4.075586
                       std       1.585751    0.692717     1.694844
                       min       2.500000    0.000000     0.500000
                       25%       5.000000    0.000000     3.000000
                       50%       6.000000    0.000000     4.000000
                       75%       7.000000    0.000000     5.000000
                       max      15.500000    3.000000    13.000000
Success                count  2106.000000  816.000000  1323.000000
                       mean      1.969611    0.279412     0.475435
                       std       0.192564    0.620997     0.499585
                       min       0.000000    0.000000     0.000000
                       25%       2.000000    0.000000     0.000000
                       50%       2.000000    0.000000     0.000000
                       75%       2.000000    0.000000     1.000000
                       max       2.000000    2.000000     1.000000

### Demographic Perspective

label                                                          0            1  \
Age at enrollment                             count  1380.000000  1666.000000   
                                              mean     25.103623    21.180672   
                                              std       8.839155     4.883300   
                                              min      18.000000    17.000000   
                                              25%      19.000000    18.000000   
                                              50%      21.000000    19.000000   
                                              75%      29.000000    21.000000   
                                              max      60.000000    62.000000   
Gender_Male                                   count  1380.000000  1666.000000   
                                              mean      0.335507     0.399760   
                                              std       0.472339     0.489996   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     0.000000   
                                              75%       1.000000     1.000000   
                                              max       1.000000     1.000000   
Marital status-alone                          count  1380.000000  1666.000000   
                                              mean      0.850000     0.968187   
                                              std       0.357201     0.175554   
                                              min       0.000000     0.000000   
                                              25%       1.000000     1.000000   
                                              50%       1.000000     1.000000   
                                              75%       1.000000     1.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Higher                 count  1380.000000  1666.000000   
                                              mean      0.003623     0.349340   
                                              std       0.060106     0.476905   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     0.000000   
                                              75%       0.000000     1.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Lower                  count  1380.000000  1666.000000   
                                              mean      0.882609     0.016807   
                                              std       0.322003     0.128585   
                                              min       0.000000     0.000000   
                                              25%       1.000000     0.000000   
                                              50%       1.000000     0.000000   
                                              75%       1.000000     0.000000   
                                              max       1.000000     1.000000   
Mother's qualification-Middle                 count  1380.000000  1666.000000   
                                              mean      0.044203     0.620048   
                                              std       0.205620     0.485520   
                                              min       0.000000     0.000000   
                                              25%       0.000000     0.000000   
                                              50%       0.000000     1.000000   
                                              75%       0.000000     1.000000   
                                              max       1.000000     1.000000   
Father's qualification-Higher  

### Final Results

final_groups                                                  0           1  \
Entry score                                   count  801.000000  543.000000   
                                              mean   128.772534  125.954328   
                                              std     14.612934   13.508768   
                                              min     98.100000   96.000000   
                                              25%    119.400000  117.550000   
                                              50%    127.500000  124.600000   
                                              75%    136.300000  132.950000   
                                              max    190.000000  190.000000   
Gender_Male                                   count  801.000000  543.000000   
                                              mean     0.274657    0.484346   
                                              std      0.446620    0.500216   
                                              min      0.000000    0.000000   
                                              25%      0.000000    0.000000   
                                              50%      0.000000    0.000000   
                                              75%      1.000000    1.000000   
                                              max      1.000000    1.000000   
Age at enrollment                             count  801.000000  543.000000   
                                              mean    20.348315   21.662983   
                                              std      4.231994    5.056019   
                                              min     17.000000   17.000000   
                                              25%     18.000000   19.000000   
                                              50%     19.000000   20.000000   
                                              75%     20.000000   22.000000   
                                              max     62.000000   53.000000   
Success                                       count  801.000000  543.000000   
                                              mean     1.977528    0.510129   
                                              std      0.156507    0.500358   
                                              min      0.000000    0.000000   
                                              25%      2.000000    0.000000   
                                              50%      2.000000    1.000000   
                                              75%      2.000000    1.000000   
                                              max      2.000000    1.000000   
Average grades                                count  801.000000  543.000000   
                                              mean    13.251288   11.977801   
                                              std      1.199481    1.825308   
                                              min      6.875000    5.000000   
                                              25%     12.400000   11.375000   
                                              50%     13.166667   12.083333   
                                              75%     14.066667   12.916667   
                                              max     18.283654   16.964286   
Average scored units                          count  801.000000  543.000000   
                                              mean     7.943196    9.251381   
                                              std      2.222095    2.408639   
                                              min      2.500000    0.000000   
                                              25%      6.500000    8.000000   
                                              50%      8.000000    9.000000   
                                              75%      8.500000   11.000000   
                                              max     17.000000   15.500000   
Average units approved                        count  801.000000  543.000000   
                                              mean